In [34]:
import logging
import os
import glob
from dotenv import load_dotenv  # 用于加载.env环境变量
from langchain.schema import Document
from langchain_community.document_loaders import PyMuPDFLoader


# from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma


from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder


# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 加载 .env 中的配置
load_dotenv()

folder_path = os.getenv("LOCAL_KNOWLEDGE_BASE_PATH", "Local_knowledge_base")
print("📂 加载路径:", folder_path)

def load_pdf_from_folder(folder_path):
    all_documents = []

    pdf_files = glob.glob(f"{folder_path}/*.pdf")
    print(f"📄 匹配到 {len(pdf_files)} 个 PDF 文件：", pdf_files)

    for filename in pdf_files:
        loader = PyMuPDFLoader(filename)
        documents = loader.load()
        all_documents.extend(documents)

    return all_documents

# 调用函数
all_documents = load_pdf_from_folder(folder_path)
print("✅ Loaded documents:", len(all_documents))

# 打印第一个文档的前100个字符（内容在 .page_content 属性中）
if len(all_documents) > 0:
    print("First document text:", all_documents[0].page_content[:100])

if len(all_documents) > 1:
    print("Second document text:", all_documents[1].page_content[:100])

📂 加载路径: /Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base
📄 匹配到 4 个 PDF 文件： ['/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/CSRF.pdf', '/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/XXE.pdf', '/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf', '/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/SSRF.pdf']
✅ Loaded documents: 307
First document text: 1
CSRF
1. 原理
2. 与XSS区别
3. 常见场景
4. 常见漏洞点
5. 漏洞危害
6. CSRF Poc 构造
7. 漏洞审计
8. 漏洞修复
9. Webgoat
跨站请求伪造（Cro
Second document text: 2

一次完整的 CSRF 攻击需要具备以下两个条件：
用户已经登录某站点，并且在浏览器中存储了登录后的 Cookie 信息。
在不注销某站点的情况下，去访问攻击者构造的站点。
例：
网站管理员添加用


切片


In [43]:
def split_documents(documents, chunk_size=1000, overlap=100):
    """
    对每个文档做分片，chunk_size控制单片长度，overlap保证上下文连贯
    输入: Document对象列表，输出: 分片list
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_documents(documents)
    
# 使用示例
# 分片处理（✅ 修复后的调用）
split_docs = split_documents(all_documents, chunk_size=1000, overlap=100)

# 打印分片信息
print("🔍 分片总数:", len(split_docs))
if split_docs:
    print("First chunk:", split_docs[0].page_content[:100])
    print("Second chunk:", split_docs[1].page_content[:100])

🔍 分片总数: 322
First chunk: 1
CSRF
1. 原理
2. 与XSS区别
3. 常见场景
4. 常见漏洞点
5. 漏洞危害
6. CSRF Poc 构造
7. 漏洞审计
8. 漏洞修复
9. Webgoat
跨站请求伪造（Cro
Second chunk: 2

一次完整的 CSRF 攻击需要具备以下两个条件：
用户已经登录某站点，并且在浏览器中存储了登录后的 Cookie 信息。
在不注销某站点的情况下，去访问攻击者构造的站点。
例：
网站管理员添加用


建立索引

In [ ]:
def build_index(documents):
    """
    Convert the sharded documents into vectors and index them
    """
    embeddings = SentenceTransformerEmbeddings(model_name="shibing624/text2vec-base-chinese")
    
    # 将文档列表转换为Document对象（确保doc是字符串类型）
    doc_objects = [Document(page_content=str(doc)) for doc in documents]
    
    # 使用Chroma创建向量索引
    index = Chroma.from_documents(doc_objects, embeddings)
    
    return index

# 使用示例：假设split_docs包含分片后的文档列表
index = build_index(split_docs)

# 打印索引信息（获取索引中文档数量）
print("Index built with", index._collection.count(), "document chunks.")

/var/folders/sd/5kzxkr7j1cq1356bqv4pt3840000gn/T/ipykernel_20737/2543798220.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="shibing624/text2vec-base-chinese")
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: shibing624/text2vec-base-chinese
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Index built with 322 document chunks.


存储在向量数据库

In [48]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document

def build_index(documents):
    """
    Convert the sharded documents into vectors and index them (in-memory, non-persistent)
    """
    embeddings = SentenceTransformerEmbeddings(model_name="shibing624/text2vec-base-chinese")
    
    # 将文档列表转换为 Document 对象
    doc_objects = [Document(page_content=str(doc)) for doc in documents]
    
    # 使用 Chroma 创建非持久化的向量索引（内存模式）
    index = Chroma.from_documents(documents=doc_objects,
                                  embedding=embeddings,
                                  collection_name="default",  # 可自定义
                                  persist_directory=None)     # 不持久化
    return index

# 使用示例（split_docs 是分片后的文本列表）
index = build_index(split_docs)

# 打印索引中的文档数量
print("Index built with", index._collection.count(), "document chunks.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: shibing624/text2vec-base-chinese


Index built with 322 document chunks.


封装

In [49]:
# ✅ 封装初始化函数（建议每次项目启动时调用）
def init_retrieval_pipeline():
    logger.info("🚀 正在初始化检索系统...")
    raw_documents = load_pdf_from_folder(folder_path)
    split_docs = split_documents(raw_documents)
    index = build_index(split_docs)
    logger.info(f"✅ 检索系统初始化完成，共加载 {len(split_docs)} 个文档分片")
    return index

召回

In [54]:
def recall_documents(query : str, index, k=5):
    """
    A similarity retrieval is performed and the k most similar documents to the query are returned
    """
    return index.similarity_search(query, k=k)


# 使用示例
query = "2024年世界职业院校分为哪些赛道?"
retrieved_docs = recall_documents(query, index, k=5)
# 打印检索到的文档内容
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1} content:", doc.page_content[:100])  # 打印每个文档的前100个字符


Document 1 content: page_content='223
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国
Document 2 content: page_content='165
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国
Document 3 content: page_content='195
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国
Document 4 content: page_content='218
附件4
2024 年世界职业院校技能大赛
赛事承办单位安全保障责任承诺书
为确保2024 年世界职业院校技能大赛（以下简称“大赛”）
顺利进行，及时排除安全隐患，防
Document 5 content: page_content='190
附件4
2024 年世界职业院校技能大赛
赛事承办单位安全保障责任承诺书
为确保2024 年世界职业院校技能大赛（以下简称“大赛”）
顺利进行，及时排除安全隐患，防


重排

In [59]:
def rerank(query, retrieved_docs, top_k: int):
    """
    Rerank the retrieved document chunks using a cross-encoder model and return the top_k contents
    """
    # 提取每个 Document 的内容
    retrieved_chunks = [doc.page_content for doc in retrieved_docs]
    
    # 创建 query-chunk 对
    pairs = [(query, chunk) for chunk in retrieved_chunks]

    # 使用 CrossEncoder 进行打分
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    scores = cross_encoder.predict(pairs)

    # 将 chunk 与分数组成元组，并按分数降序排序
    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    # 返回 Top-K 重排后的 chunk 文本
    return [chunk for chunk, _ in scored_chunks][:top_k]

# 使用示例
query = "2024年世界职业院校分为哪些赛道?"
retrieved_docs = recall_documents(query, index, k=5)

# 重排前查看召回结果（可选）
print("🔍 初始召回结果：")
for i, doc in enumerate(retrieved_docs):
    print(f"[召回{i+1}] {doc.page_content[:100]}...")

# 对召回结果进行重排
reranked_chunks = rerank(query, retrieved_docs, top_k=3)

# 输出最终重排后的内容
print("\n🏆 Top-3 重排后结果：")
for i, chunk in enumerate(reranked_chunks):
    print(f"[{i+1}] {chunk}\n")

🔍 初始召回结果：
[召回1] page_content='223
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国...
[召回2] page_content='165
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国...
[召回3] page_content='195
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国...
[召回4] page_content='218
附件4
2024 年世界职业院校技能大赛
赛事承办单位安全保障责任承诺书
为确保2024 年世界职业院校技能大赛（以下简称“大赛”）
顺利进行，及时排除安全隐患，防...
[召回5] page_content='190
附件4
2024 年世界职业院校技能大赛
赛事承办单位安全保障责任承诺书
为确保2024 年世界职业院校技能大赛（以下简称“大赛”）
顺利进行，及时排除安全隐患，防...


INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


🏆 Top-3 重排后结果：
[1] page_content='195
（三）《赛事指南》封三要求
封三为大赛简介（中英文双语版）。
1. 2024 年世界职业院校技能大赛（简介）
全国职业院校技能大赛（简称大赛）是教育部牵头、联
合国家部委和事业组织举办的一项公益性、国际性职业院校
综合技能竞赛，是我国职业教育一项重大制度设计和创新，
是举办历史最久、联合主办部委最全、赛项数量最多的世界
性综合技能赛事。大赛始终坚持“以赛促教、以赛促学”的
理念，自2008 年以来已连续举办16 届，办赛规模和影响力
持续提升，大赛成果不断转化为专业人才培养方案、课程和
教学内容，有效推进了校企合作和实习实训基地建设，在引
领职业教育“三教”改革、提高技术技能人才培养质量、促
进高质量就业、服务经济社会发展、助力中外职业教育交流
合作等方面发挥了重要作用，已经成为广大职教师生展示风
采、追梦圆梦的重要舞台和中国职业教育的靓丽品牌。
党的二十大报告对职业教育在全面建设社会主义现代
化国家新征程中的作用提出了新要求，强国建设赋予了职业
教育新的时代使命。为强化大赛综合育人功能，提升赛事实
施科学水平，打造职业教育国际品牌，经大赛组委会研究，
决定将“全国职业院校技能大赛”升级为“世界职业院校技
能大赛”，优化大赛体制机制，助力培育新质生产力、建设
现代化产业体系、培养具备综合技能意识和实践动手能力的
高技能人才。' metadata={'producer': '', 'creator': 'WPS 文字', 'creationdate': '2024-09-30T16:02:34+08:00', 'source': '/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf', 'file_path': '/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf', 'total_pages': 255, 'format': 'PDF 1.7', 'title': '', 'author': '26742', 'subject': '', 'k

生成

In [ ]:
# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
def generate_answer(query, reranked_chunks):
    """
    基于查询和重排后的 chunk 文本生成最终答案
    """
    documents_content = "\n".join(reranked_chunks)

    prompt = (
        "你是一个智能助手，请根据以下内容回答用户问题：\n\n"
        f"{documents_content}\n\n用户提问：{query}\n\n请用简洁准确的语言作答："
    )

    llm = ChatOllama(base_url="http://localhost:11434", model="llama3.1:8b")
    print(llm)

    try:
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        print(f"❌ Error generating answer: {e}")
        return None
    

In [97]:

# 示例流程
query = "2024年世界职业技能大赛总决赛分为哪三个阶段进行？"


query = "2024年世界职业技能大赛总决赛分为哪三个阶段进行？"

# 第一步：召回
retrieved_docs = recall_documents(query, index, k=5)
print(retrieved_docs)

print("------")
# 第二步：重排
reranked_chunks = rerank(query, retrieved_docs, top_k=3)
print(reranked_chunks)

print("------")
# 第三步：生成答案
answer = generate_answer(query, reranked_chunks)
print(answer)


# 输出
if answer:
    print("✅ Generated answer:", answer)
else:
    print("❌ Failed to generate an answer.")

[Document(metadata={}, page_content='page_content=\'195\n（三）《赛事指南》封三要求\n封三为大赛简介（中英文双语版）。\n1. 2024 年世界职业院校技能大赛（简介）\n全国职业院校技能大赛（简称大赛）是教育部牵头、联\n合国家部委和事业组织举办的一项公益性、国际性职业院校\n综合技能竞赛，是我国职业教育一项重大制度设计和创新，\n是举办历史最久、联合主办部委最全、赛项数量最多的世界\n性综合技能赛事。大赛始终坚持“以赛促教、以赛促学”的\n理念，自2008 年以来已连续举办16 届，办赛规模和影响力\n持续提升，大赛成果不断转化为专业人才培养方案、课程和\n教学内容，有效推进了校企合作和实习实训基地建设，在引\n领职业教育“三教”改革、提高技术技能人才培养质量、促\n进高质量就业、服务经济社会发展、助力中外职业教育交流\n合作等方面发挥了重要作用，已经成为广大职教师生展示风\n采、追梦圆梦的重要舞台和中国职业教育的靓丽品牌。\n党的二十大报告对职业教育在全面建设社会主义现代\n化国家新征程中的作用提出了新要求，强国建设赋予了职业\n教育新的时代使命。为强化大赛综合育人功能，提升赛事实\n施科学水平，打造职业教育国际品牌，经大赛组委会研究，\n决定将“全国职业院校技能大赛”升级为“世界职业院校技\n能大赛”，优化大赛体制机制，助力培育新质生产力、建设\n现代化产业体系、培养具备综合技能意识和实践动手能力的\n高技能人才。\' metadata={\'producer\': \'\', \'creator\': \'WPS 文字\', \'creationdate\': \'2024-09-30T16:02:34+08:00\', \'source\': \'/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf\', \'file_path\': \'/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf\', \'total_pages\': 255, \'form

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
Batches: 100%|██████████| 1/1 [00:00<00:00, 31.56it/s]


['page_content=\'196\n2024 年大赛充分考虑国家产业发展方向、重点行业、民\n生福祉密切相关产业和生产实践需要，共设置42 个赛道。\n大赛总决赛分争夺赛、排位赛、冠军总决赛三个阶段进行，\n其中争夺赛在北京、天津、河北、山西、内蒙古、辽宁、吉\n林、黑龙江、上海、江苏、浙江、安徽、福建、江西、山东、\n河南、湖北、湖南、广东、广西、海南、重庆、四川、贵州、\n云南、西藏、陕西、甘肃、青海、宁夏、新疆、新疆生产建\n设兵团等32 个赛区举行，排位赛、冠军总决赛在天津赛区\n举行。\n教育部将深入贯彻落实习近平总书记关于职业教育重\n要指示批示精神和党中央、国务院关于职业教育改革发展的\n决策部署，以“技炫青春能创未来”为主题，秉持“精彩、\n公平、专业、开放”的办赛理念，立足国内、放眼世界，努\n力将大赛办成具有中国特色、世界水平的技能大赛，展示我\n国职业教育成就和水平，搭建世界职业教育互学互鉴、友好\n交流平台，推动培养更多适应新技术变革和产业发展需要的\n高技能人才、能工巧匠、大国工匠。\n2. 2024 World Vocational College Skills Competition\n（Introduction）\nNational Vocational College Skills Competition (NVCSC)\nis initiated by the Ministry of Education (MOE) of the People’s\nRepublic of China, with the joint efforts of relevant national\nministries and commissions, as well as public institutions and\' metadata={\'producer\': \'\', \'creator\': \'WPS 文字\', \'creationdate\': \'2024-09-30T16:02:34+08:00\', \'source\': \'/Users/queen/Documents/VSCode/llm_retrieval/Local_knowledge_base/2024年世界职业院校技能大赛制度汇编.pdf\', \'fi